In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb


from path_setup import setup_paths
setup_paths()

In [2]:
from dataset_transformer import BaseDatasetTransform
from base_model_train import BaseModelTrain
from model_evaluator import ModelEvaluator

In [3]:
df = pd.read_csv('../datasets/cosmetics.csv')

In [4]:
df.head()

,Label,Brand,Name,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive
0,Moisturizer,LA MER,Crème de la Mer,175,4.1,"Algae (Seaweed) Extract, Mineral Oil, Petrolat...",1,1,1,1,1
1,Moisturizer,SK-II,Facial Treatment Essence,179,4.1,"Galactomyces Ferment Filtrate (Pitera), Butyle...",1,1,1,1,1
2,Moisturizer,DRUNK ELEPHANT,Protini™ Polypeptide Cream,68,4.4,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",1,1,1,1,0
3,Moisturizer,LA MER,The Moisturizing Soft Cream,175,3.8,"Algae (Seaweed) Extract, Cyclopentasiloxane, P...",1,1,1,1,1
4,Moisturizer,IT COSMETICS,Your Skin But Better™ CC+™ Cream with SPF 50+,38,4.1,"Water, Snail Secretion Filtrate, Phenyl Trimet...",1,1,1,1,1


In [5]:
import re
df['Name'] = df['Name'].str.lower().apply(lambda x: re.sub(r'[^a-z\s]', '', x))

In [6]:
df.head()

,Label,Brand,Name,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive
0,Moisturizer,LA MER,crme de la mer,175,4.1,"Algae (Seaweed) Extract, Mineral Oil, Petrolat...",1,1,1,1,1
1,Moisturizer,SK-II,facial treatment essence,179,4.1,"Galactomyces Ferment Filtrate (Pitera), Butyle...",1,1,1,1,1
2,Moisturizer,DRUNK ELEPHANT,protini polypeptide cream,68,4.4,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",1,1,1,1,0
3,Moisturizer,LA MER,the moisturizing soft cream,175,3.8,"Algae (Seaweed) Extract, Cyclopentasiloxane, P...",1,1,1,1,1
4,Moisturizer,IT COSMETICS,your skin but better cc cream with spf,38,4.1,"Water, Snail Secretion Filtrate, Phenyl Trimet...",1,1,1,1,1


In [7]:
df['Label'].unique()

array(['Moisturizer', 'Cleanser', 'Treatment', 'Face Mask', 'Eye cream',
       'Sun protect'], dtype=object)

In [8]:
from collections import Counter

c = Counter(''.join(df['Name']).split())
c.most_common(10)


[('spf', 212),
 ('eye', 177),
 ('spectrum', 115),
 ('broad', 114),
 ('face', 71),
 ('cream', 70),
 ('mask', 67),
 ('cleansing', 65),
 ('with', 62),
 ('skin', 58)]

Юзать признак name не надо 

In [9]:
"""
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import numpy as np

descriptions = df['Ingredients'].tolist()
model = SentenceTransformer('all-MiniLM-L6-v2')

embedding_batch_size = 128  


embeddings = []


for i in tqdm(range(0, len(descriptions), embedding_batch_size)):
    batch = descriptions[i:i + embedding_batch_size]
    batch_embeddings = model.encode(batch)
    embeddings.extend(batch_embeddings)

embeddings = np.array(embeddings)
"""

"\nfrom sentence_transformers import SentenceTransformer\nfrom tqdm import tqdm\nimport numpy as np\n\ndescriptions = df['Ingredients'].tolist()\nmodel = SentenceTransformer('all-MiniLM-L6-v2')\n\nembedding_batch_size = 128  \n\n\nembeddings = []\n\n\nfor i in tqdm(range(0, len(descriptions), embedding_batch_size)):\n    batch = descriptions[i:i + embedding_batch_size]\n    batch_embeddings = model.encode(batch)\n    embeddings.extend(batch_embeddings)\n\nembeddings = np.array(embeddings)\n"

In [10]:
"""
col = [f'{i+1}_feature' for i in range(embeddings.shape[1])]

tfidf_df = pd.DataFrame(embeddings, columns=col)

df = pd.concat([df, tfidf_df], axis=1)

df
"""

"\ncol = [f'{i+1}_feature' for i in range(embeddings.shape[1])]\n\ntfidf_df = pd.DataFrame(embeddings, columns=col)\n\ndf = pd.concat([df, tfidf_df], axis=1)\n\ndf\n"

In [11]:

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(tokenizer=lambda x: x.split(', '), max_features=100330)

X = vectorizer.fit_transform(df['Ingredients'].fillna(''))

col = [f'{i+1}_feature' for i in range(X.shape[1])]
tfidf_df = pd.DataFrame(X.toarray(), columns=col)

df = pd.concat([df, tfidf_df], axis=1)

df

c:\Users\Dmitriy.Burnashev\AppData\Local\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,Label,Brand,Name,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,...,6824_feature,6825_feature,6826_feature,6827_feature,6828_feature,6829_feature,6830_feature,6831_feature,6832_feature,6833_feature
0,Moisturizer,LA MER,crme de la mer,175,4.1,"Algae (Seaweed) Extract, Mineral Oil, Petrolat...",1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Moisturizer,SK-II,facial treatment essence,179,4.1,"Galactomyces Ferment Filtrate (Pitera), Butyle...",1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Moisturizer,DRUNK ELEPHANT,protini polypeptide cream,68,4.4,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Moisturizer,LA MER,the moisturizing soft cream,175,3.8,"Algae (Seaweed) Extract, Cyclopentasiloxane, P...",1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Moisturizer,IT COSMETICS,your skin but better cc cream with spf,38,4.1,"Water, Snail Secretion Filtrate, Phenyl Trimet...",1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467,Sun protect,KORRES,yoghurt nourishing fluid veil face sunscreen b...,35,3.9,"Water, Alcohol Denat., Potassium Cetyl Phospha...",1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1468,Sun protect,KATE SOMERVILLE,daily deflector waterlight broad spectrum spf ...,48,3.6,"Water, Isododecane, Dimethicone, Butyloctyl Sa...",0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1469,Sun protect,VITA LIBERATA,self tan dry oil spf,54,3.5,"Water, Dihydroxyacetone, Glycerin, Sclerocarya...",0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1470,Sun protect,ST. TROPEZ TANNING ESSENTIALS,pro light self tan bronzing mist,20,1.0,"Water, Dihydroxyacetone, Propylene Glycol, PPG...",0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
df = df.drop(['Ingredients','Name'],axis=1)

In [13]:
target_columns = ['Combination','Dry','Normal','Oily','Sensitive']

In [14]:
bdf = BaseDatasetTransform(df,target=target_columns)

In [15]:
df,categorical_features = bdf.fit_transform()

Series([], dtype: int64)

В колонке(ах) ['Combination', 'Dry', 'Normal', 'Oily', 'Sensitive'] нет пропущенных значений
-------------------------------------------
В наборе данных нет пропущенных значений
-------------------------------------------
Информация о колонках в датасете

Категориальные колонки:
Index(['Label', 'Brand'], dtype='object')

Числовые колонки:
Index(['Price', 'Rank', 'Combination', 'Dry', 'Normal', 'Oily', 'Sensitive',
       '1_feature', '2_feature', '3_feature',
       ...
       '6824_feature', '6825_feature', '6826_feature', '6827_feature',
       '6828_feature', '6829_feature', '6830_feature', '6831_feature',
       '6832_feature', '6833_feature'],
      dtype='object', length=6840)


In [16]:
X_train, X_test, y_train, y_test  = bdf.get_train_test_split()

Количество значений целевой переменной по категориям:
Количество значений комбинаций целевых переменных по категориям:
1_1_1_1_1    662
0_0_0_0_0    471
1_1_1_1_0    145
1_1_1_0_1     44
1_0_1_1_0     34
1_0_1_1_1     25
1_0_0_1_0     20
1_1_1_0_0     19
0_1_1_0_1      9
0_1_1_0_0      8
1_0_1_0_0      8
0_1_0_0_0      5
0_1_0_0_1      4
1_1_0_0_0      3
0_0_0_0_1      2
0_0_0_1_1      2
1_0_1_0_1      2
0_0_1_1_0      2
1_1_0_0_1      2
0_0_1_0_1      1
1_1_0_1_1      1
1_0_0_1_1      1
0_1_1_1_0      1
0_1_0_1_1      1
Name: count, dtype: int64
Следует ли выполнить стратифицированное раздеение на обучающую и тестовую выборку? y/n
Разделение датасета выполнено успешно


In [17]:
from sklearn.multioutput import MultiOutputClassifier


lgbm_model = MultiOutputClassifier(lgb.LGBMClassifier(verbose=-1))

columns_to_delete = []  

unique_classes = len(set(y_train))
print(unique_classes)

model_train = BaseModelTrain(
    model=lgbm_model,
    columns_to_delete=columns_to_delete,
    categorical_features=categorical_features,
    boosting_type='gbdt',
    #objective='binary',  
    #metric='binary_error' 
)

5


In [18]:
metrics = ['accuracy', 'precision', 'recall', 'f1_macro']
evaluator = ModelEvaluator(model_trainer=model_train, metrics=metrics,base_multitarget=True)

evaluator.fit(X_train, y_train)

ModelEvaluator(base_multitarget=True,
               metrics=['accuracy', 'precision', 'recall', 'f1_macro'],
               model_trainer=BaseModelTrain(categorical_features=['Label',
                                                                  'Brand'],
                                            columns_to_delete=[],
                                            model=MultiOutputClassifier(estimator=LGBMClassifier(verbose=-1))))

In [19]:
evaluator.evaluate_to_dataframe(X_test, y_test)

,accuracy,precision,recall,f1_macro,Targets
0,0.701357,0.669822,0.643939,0.64951,Combination
1,0.674208,0.660261,0.648618,0.650991,Dry
2,0.692308,0.662096,0.638465,0.643184,Normal
3,0.674208,0.661968,0.645208,0.64734,Oily
4,0.671946,0.672969,0.671946,0.67146,Sensitive


In [20]:
evaluator.evaluate_to_dataframe(X_test, y_test)

,accuracy,precision,recall,f1_macro,Targets
0,0.701357,0.669822,0.643939,0.64951,Combination
1,0.674208,0.660261,0.648618,0.650991,Dry
2,0.692308,0.662096,0.638465,0.643184,Normal
3,0.674208,0.661968,0.645208,0.64734,Oily
4,0.671946,0.672969,0.671946,0.67146,Sensitive


In [21]:
'''
accuracy	precision	recall	f1_macro	Targets
0	0.701357	0.669822	0.643939	0.64951	Combination
1	0.674208	0.660261	0.648618	0.650991	Dry
2	0.692308	0.662096	0.638465	0.643184	Normal
3	0.674208	0.661968	0.645208	0.64734	Oily
4	0.671946	0.672969	0.671946	0.67146	Sensitive
'''

'\naccuracy\tprecision\trecall\tf1_macro\tTargets\n0\t0.701357\t0.669822\t0.643939\t0.64951\tCombination\n1\t0.674208\t0.660261\t0.648618\t0.650991\tDry\n2\t0.692308\t0.662096\t0.638465\t0.643184\tNormal\n3\t0.674208\t0.661968\t0.645208\t0.64734\tOily\n4\t0.671946\t0.672969\t0.671946\t0.67146\tSensitive\n'